# 0. Imports, libraries and rusable functions

In [1]:
from project_imports import *
import use_gpu

Project libraries imported!
GPU: NVIDIA GeForce RTX 4070 Ti SUPER is available.
Device:cuda


# 1. Global Variables

In [2]:
## Arguments and global vriables
dataset_name="ReClor"
pretrained_model_name = "microsoft/deberta-v3-base"
normalized_model_name = pretrained_model_name.replace("/", "-")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
data_collator = DefaultDataCollator()
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
pad_on_right = right_padding = tokenizer.padding_side == 'right'
global_counter = 0
traing_answer_mismatches = []
logger = logging.getLogger(__name__)

c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


# 2. Prepare the AR-LSAT Dataset 

In [3]:
# Load the combined dataset
combined_dataset = load_from_disk('cleaned_dataset')

combined_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 1072514
    })
    validation: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 118521
    })
    test: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 200566
    })
})

In [4]:
# Filter the dataset to only include AR-LSAT data
reclor_train = combined_dataset['train'].filter(lambda x: x['Source Dataset'] == 'ReClor')
reclor_val = combined_dataset['validation'].filter(lambda x: x['Source Dataset'] == 'ReClor')
reclor_test = combined_dataset['test'].filter(lambda x: x['Source Dataset'] == 'ReClor')


# Concatenate test data into the training data
reclor_train = concatenate_datasets([reclor_train, reclor_test])

In [5]:
# Preprocessing function for multiple-choice tasks
def mcqa_preprocess_function(examples):
    num_choices = num_choices = len(examples['Options'][0])    
    first_sentences = [[context] * num_choices for context in examples['Context']]  # Repeat context for each option
    question_headers = examples['Question']
    options_list = examples['Options']
    
    second_sentences = []
    for question, options in zip(question_headers, options_list):
        # Combine question with each option
        second_sentences.append([f"{question} {option}" for option in options])
    
    # Flatten the lists
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    
    # Tokenize the inputs
    tokenized_examples = tokenizer(
        first_sentences,
        second_sentences,
        truncation=True,
        max_length=512,
        padding='max_length',
    )
    
    # Un-flatten the tokenized inputs to have shape (num_examples, num_choices, seq_length)
    tokenized_inputs = {k: [v[i:i + num_choices] for i in range(0, len(v), num_choices)] for k, v in tokenized_examples.items()}
    
    # Labels
    tokenized_inputs["labels"] = examples["Label"]
    
    return tokenized_inputs


In [6]:
def mcqa_preprocess_function(examples):
    # Determine the maximum number of choices
    max_num_choices = 5  
    contexts = examples['Context']
    questions = examples['Question']
    options_list = examples['Options']
    labels = examples['Label']
    
    first_sentences = []
    second_sentences = []
    labels_adjusted = []
    
    for context, question, options, label in zip(contexts, questions, options_list, labels):
        num_choices = len(options)
        # Pad options to have max_num_choices
        if num_choices < max_num_choices:
            options += [''] * (max_num_choices - num_choices)
        first_sentences.append([context] * max_num_choices)
        second_sentences.append([f"{question} {option}" for option in options])
        labels_adjusted.append(label)
    
    # Flatten the lists
    first_sentences = [item for sublist in first_sentences for item in sublist]
    second_sentences = [item for sublist in second_sentences for item in sublist]
    
    # Tokenize the inputs
    tokenized_examples = tokenizer(
        first_sentences,
        second_sentences,
        truncation=True,
        max_length=512,
        padding='max_length',
    )
    
    # Un-flatten to shape (num_examples, max_num_choices, seq_length)
    tokenized_inputs = {
        k: [v[i:i + max_num_choices] for i in range(0, len(v), max_num_choices)]
        for k, v in tokenized_examples.items()
    }
    
    # Labels
    tokenized_inputs["labels"] = labels_adjusted
    
    return tokenized_inputs




In [7]:

# Apply the preprocessing function to the combined datasets
encoded_reclor_train = reclor_train.map(mcqa_preprocess_function, batched=True)
encoded_reclor_val = reclor_val.map(mcqa_preprocess_function, batched=True)
encoded_reclor_test = reclor_test.map(mcqa_preprocess_function, batched=True)

In [8]:
# Set the format of the datasets to PyTorch tensors
encoded_reclor_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_reclor_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_reclor_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


def get_train_encoded():
    return encoded_reclor_train

def get_val_encoded():
    return encoded_reclor_val

def get_test_encoded():
    return encoded_reclor_test


# 3. Reusable Functions

In [9]:
# Load the accuracy metric
accuracy = evaluate.load('accuracy')

# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)['accuracy']
    f1 = f1_score(labels, predictions, average='weighted')
    return {'eval_accuracy': acc, 'eval_f1': f1}

In [10]:
def create_training_args(run_name="Default-Run", num_train_epochs=3, learning_rate=4.92e-05, batch_size=3):
    """
    Generates training arguments for training a machine learning model.

    Parameters:
    - dataset_name (str): The name of the dataset.
    - run_name (str): The name of the run, useful for logging and saving models.
    - model_name (str): The name of the model, typically including its configuration.
    - num_train_epochs (int): The number of epochs to train for.
    - learning_rate (float): The learning rate for training.
    - batch_size (int): The batch size used for training.

    Returns:
    - TrainingArguments: A configured TrainingArguments instance.
    """    
    output_dir = f"./{dataset_name}/{run_name}/{normalized_model_name}"

    if run_name=="Optuna":
        training_args = TrainingArguments(
        output_dir=output_dir,
        report_to="none",  # Disable all integrations
        overwrite_output_dir=True,
        metric_for_best_model='eval_accuracy',
        greater_is_better=True,
        load_best_model_at_end=True,
        save_total_limit=3,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=num_train_epochs,
        learning_rate=1.5807103066634623e-05,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,        
        warmup_ratio = 0.5994150649377659,
        weight_decay=0.12506835879573128,
        adam_beta1=0.8136227307274486,
        adam_beta2=0.9924116710027883,
        adam_epsilon=1.9858068243318367e-07,
        lr_scheduler_type='cosine_with_restarts',
        fp16=True,  # Enable mixed-precision training
    )
    else:
        training_args = TrainingArguments(
            output_dir=output_dir,
            report_to="none",  # Disable all integrations
            overwrite_output_dir=True,
            metric_for_best_model='eval_accuracy',
            greater_is_better=True,
            load_best_model_at_end=True,
            save_total_limit=3,
            eval_strategy="epoch",
            save_strategy="epoch",
            num_train_epochs=num_train_epochs,
            learning_rate=learning_rate,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=4,
            warmup_steps=398,
            weight_decay=0.194,
            adam_beta1=0.837,
            adam_beta2=0.997,
            adam_epsilon=5.87e-07,
            lr_scheduler_type='cosine',
            fp16=True,  # Enable mixed-precision training
        )
    
    return training_args


In [11]:
class AdvancedEarlyStoppingCallback(TrainerCallback):
    """
    A callback to stop training when either the performance falls below a certain threshold
    or if there is no improvement over a set number of epochs.
    """
    def __init__(self, metric_name, patience):
        self.metric_name = metric_name
        self.patience = patience        
        self.best_score = None
        self.no_improve_epochs = 0
        self.config_file = "early_stopping_config.json"  # Config file for early stopping values

    def read_early_stopping_config(self):
        """
        Reads the early stopping configuration from the file system.
        Returns the configuration as a dictionary.
        """
        if os.path.exists(self.config_file):
            with open(self.config_file, 'r') as file:
                config = json.load(file)
            return config
        else:
            raise FileNotFoundError(f"Config file not found: {self.config_file}")
    def reset_manual_stop_flag(self):
        """
        Resets the manual stop flag to False in the early stopping config file.
        """
        config = self.read_early_stopping_config()
        config['manual_stop'] = False
        with open(self.config_file, 'w') as file:
            json.dump(config, file, indent=4)

    def on_evaluate(self, args, state, control, **kwargs):
        metric_value = kwargs['metrics'].get(self.metric_name)

        if self.best_score is None or metric_value > self.best_score:
            self.best_score = metric_value
            self.no_improve_epochs = 0
        else:
            self.no_improve_epochs += 1

        # Check if no improvement has been seen over the allowed patience
        if self.no_improve_epochs >= self.patience:
            control.should_training_stop = True
            print(f"Stopping training: No improvement in {self.metric_name} for {self.patience} epochs")


        # Read the early stopping configuration
        config = self.read_early_stopping_config()
        min_accuracy = config.get("min_accuracy", 0.35)                
        num_epochs_min_acc = config.get("num_epochs_min_acc", 2)  
        max_variance = config.get("max_variance", 0.2)  

        # Check if performance is below the threshold
        if metric_value < min_accuracy:
            control.should_training_stop = True
            print(f"Stopping training: {self.metric_name} below manual min_acc of {min_accuracy}")

         # Manual stop from config
        if config.get("manual_stop", False):
            control.should_training_stop = True
            print(f"Manual early stopping triggered!!")
            self.reset_manual_stop_flag()  # Reset the flag for future runs
            


In [12]:
def model_init(model_name=pretrained_model_name):
    return AutoModelForMultipleChoice.from_pretrained(model_name)

In [13]:
def create_trainer(model_name=pretrained_model_name,run_name="Default-Run", num_train_epochs=3, learning_rate=4.92e-05, batch_size=4):
    trainer = Trainer(
        model=model_init(model_name),
        args=create_training_args(run_name=run_name, num_train_epochs=num_train_epochs, learning_rate=learning_rate, batch_size=batch_size),
        train_dataset=get_train_encoded(),
        eval_dataset=get_val_encoded(),
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[AdvancedEarlyStoppingCallback(metric_name='eval_accuracy', patience=1)]
    )
    
    return trainer


In [14]:
# Optuna objective function for hyperparameter tuning
def objective(trial):
    # Clear any cached memory to start fresh for each trial
    torch.cuda.empty_cache()
    gc.collect()

    
    #model_name = trial.suggest_categorical('model_name', [pretrained_model_name, "./squad-trained-model", './MCQA-Combined/Optuna/trial_5/checkpoint-22865', './MCQA-Combined/Optuna/trial_0/checkpoint-30485','./MCQA-Combined/Optuna/trial_6/checkpoint-30485' ]) 
    model_name ="./MCQA-Combined/Optuna/trial_5/checkpoint-22865"
    learning_rate = trial.suggest_float('learning_rate', 1e-7, 1e-4, log=True)
    batch_size = trial.suggest_categorical('batch_size', [3, 4])
    #warmup_steps = trial.suggest_int('warmup_steps', 0, 1000)
    warmup_ratio= trial.suggest_float('warmup_ratio', 0.0, 1.0)
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.25)
    adam_beta1 = trial.suggest_float('adam_beta1', 0.8, 0.95)
    adam_beta2 = trial.suggest_float('adam_beta2', 0.990, 0.999)
    adam_epsilon = trial.suggest_float('adam_epsilon', 1e-8, 1e-6)
    lr_scheduler_type = trial.suggest_categorical('lr_scheduler_type', ['linear', 'cosine', 'cosine_with_restarts']) #,'constant_with_warmup'   
    

    output_dir = f"./{dataset_name}/Optuna3/trial_{trial.number}"

    training_args = TrainingArguments(
        output_dir=output_dir,
        report_to="none",  # Disable all integrations
        overwrite_output_dir=True,
        metric_for_best_model='eval_accuracy',
        greater_is_better=True,
        load_best_model_at_end=True,
        save_total_limit=3,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=30,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_ratio=warmup_ratio,
        weight_decay=weight_decay,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        lr_scheduler_type=lr_scheduler_type,
        fp16=True,  # Enable mixed-precision training
    ) 
    
    # Print trial parameters
    print(f"Current Trial {trial.number} parameters: {trial.params}")
    
    trainer = Trainer(
        model=model_init(model_name),
        args=training_args,
        train_dataset=get_train_encoded(),
        eval_dataset=get_val_encoded(),
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[AdvancedEarlyStoppingCallback(metric_name='eval_accuracy', patience=1)]
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
        
    torch.cuda.empty_cache()  # Clear cache after evaluation
    gc.collect()  # Collect garbage

    return eval_results['eval_accuracy']


# 4. Fine-tuning DeBERTa on MCQA task (AR-LSAT Dataset)

## 4.1 Evaluate Vanilla DeBERTa (Acc= 27.6%)

In [17]:
# Load the model
model = AutoModelForMultipleChoice.from_pretrained(pretrained_model_name)

# Create the Trainer
trainer = create_trainer()

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print(f"Test Results: {test_results}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: mzak071 (COMPSCI714). Use `wandb login --relogin` to force relogin


Test Results: {'eval_accuracy': 0.276, 'eval_f1': 0.2767257999541722, 'eval_loss': 1.3862577676773071, 'eval_model_preparation_time': 0.002, 'eval_runtime': 15.3247, 'eval_samples_per_second': 32.627, 'eval_steps_per_second': 32.627}


## 4.2 Fine-Tune and Evaluate Vanilla DeBERTa (Acc=38.2%)

In [19]:
# Load the model
model = AutoModelForMultipleChoice.from_pretrained(pretrained_model_name)
# Create the Trainer
trainer = create_trainer()

# Train the model
trainer.train()
# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print(f"Test Results: {test_results}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.386800,1.385791,0.342685,0.342227
2,1.391000,1.386014,0.296593,0.297131
3,1.305200,1.325571,0.372745,0.370938


Test Results: {'eval_accuracy': 0.382, 'eval_f1': 0.38155725956214587, 'eval_loss': 1.3181953430175781, 'eval_runtime': 14.3702, 'eval_samples_per_second': 34.794, 'eval_steps_per_second': 34.794, 'epoch': 3.0}


## 4.3 Evaluate SQUAD DeBERTa (Acc=25.2%)

In [21]:
path = "./squad-trained-model"
model =  AutoModelForMultipleChoice.from_pretrained(path)
# Create the Trainer
trainer = create_trainer(run_name="Squad-Run")
# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print(f"Test Results: {test_results}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at ./squad-trained-model and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Test Results: {'eval_accuracy': 0.252, 'eval_f1': 0.2527663131875577, 'eval_loss': 1.386056661605835, 'eval_model_preparation_time': 0.002, 'eval_runtime': 14.2706, 'eval_samples_per_second': 35.037, 'eval_steps_per_second': 35.037}


## 4.4 Fine-Tune and Evaluate  SQUAD DeBERTa (Acc=46.8%)


In [24]:
path = "./squad-trained-model"
model =  AutoModelForMultipleChoice.from_pretrained(path)
# Create the Trainer
trainer = create_trainer(run_name="Squad-Run")

# Train the model
trainer.train()

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print(f"Test Results: {test_results}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at ./squad-trained-model and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: mzak071 (COMPSCI714). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.372300,1.320340,0.404810,0.404691
2,1.119300,1.094201,0.537074,0.537099
3,0.611000,1.609691,0.551102,0.550685


Test Results: {'eval_accuracy': 0.468, 'eval_f1': 0.4681988749254519, 'eval_loss': 1.9973200559616089, 'eval_runtime': 14.2601, 'eval_samples_per_second': 35.063, 'eval_steps_per_second': 35.063, 'epoch': 3.0}


## 4.5 Evaluate Others

In [43]:
path = "./LogiQA/Squad-Run/microsoft-deberta-v3-base/checkpoint-12567"
model =  AutoModelForMultipleChoice.from_pretrained(path)
# Create the Trainer
trainer = create_trainer(run_name="LogiQA-Run")

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print(f"Test Results: {test_results}")

C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Test Results: {'eval_accuracy': 0.4, 'eval_f1': 0.4006366456726491, 'eval_loss': 1.3058828115463257, 'eval_model_preparation_time': 0.0, 'eval_runtime': 14.525, 'eval_samples_per_second': 34.423, 'eval_steps_per_second': 34.423}


In [10]:
path = "./MCQA-Combined/Squad-Run/microsoft-deberta-v3-base/checkpoint-18291"
model =  AutoModelForMultipleChoice.from_pretrained(path)
# Create the Trainer
trainer = create_trainer(run_name="MCQA-Combined-Run", batch_size=3,  num_train_epochs=1)

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/500 [00:00<?, ?it/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mzak071 (COMPSCI714). Use `wandb login --relogin` to force relogin


Test Results:
Accuracy: 0.3340
F1 Score: 0.3335


In [14]:
path = "./MCQA-Combined/Squad-Run/microsoft-deberta-v3-base/checkpoint-18291"
model =  AutoModelForMultipleChoice.from_pretrained(path)
# Create the Trainer
trainer = create_trainer(run_name="MCQA-Combined-Run", batch_size=4,  num_train_epochs=6)

# Train the model
trainer.train()

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/6210 [00:00<?, ?it/s]

{'loss': 1.3947, 'grad_norm': 1.515221357345581, 'learning_rate': 4.9164785591849574e-05, 'epoch': 0.48}
{'loss': 1.3877, 'grad_norm': 1.8046776056289673, 'learning_rate': 4.7921767366118935e-05, 'epoch': 0.97}


  0%|          | 0/499 [00:00<?, ?it/s]

{'eval_accuracy': 0.2985971943887776, 'eval_f1': 0.2984327577081756, 'eval_loss': 1.385642409324646, 'eval_runtime': 17.1848, 'eval_samples_per_second': 29.037, 'eval_steps_per_second': 29.037, 'epoch': 1.0}
{'loss': 1.3857, 'grad_norm': 1.6376599073410034, 'learning_rate': 4.4985562567144186e-05, 'epoch': 1.45}
{'loss': 1.3774, 'grad_norm': 1.8705437183380127, 'learning_rate': 4.0569342969768665e-05, 'epoch': 1.93}


...

{'eval_accuracy': 0.23647294589178355, 'eval_f1': 0.18601877501047798, 'eval_loss': 1.3865934610366821, 'eval_runtime': 18.016, 'eval_samples_per_second': 27.698, 'eval_steps_per_second': 27.698, 'epoch': 5.0}
{'loss': 1.3697, 'grad_norm': 2.079101800918579, 'learning_rate': 1.8044730062000698e-06, 'epoch': 5.31}
{'loss': 1.3723, 'grad_norm': 2.223784923553467, 'learning_rate': 1.6286698398285957e-07, 'epoch': 5.8}


  0%|          | 0/499 [00:00<?, ?it/s]

{'eval_accuracy': 0.23246492985971945, 'eval_f1': 0.08845007087345423, 'eval_loss': 1.3867168426513672, 'eval_runtime': 17.7611, 'eval_samples_per_second': 28.095, 'eval_steps_per_second': 28.095, 'epoch': 6.0}
{'train_runtime': 5386.1301, 'train_samples_per_second': 4.61, 'train_steps_per_second': 1.153, 'train_loss': 1.3788943673101601, 'epoch': 6.0}


  0%|          | 0/500 [00:00<?, ?it/s]

Test Results:
Accuracy: 0.3340
F1 Score: 0.3315


# End of NoteBook

## First Tuninig

In [56]:
# Create the Trainer
#trainer = create_trainer(model_name="./MCQA-Combined/Optuna/trial_5/checkpoint-22865", run_name="Optuna")  - Official Score 51.22%
trainer = create_trainer(model_name="./ReClor/Optuna/trial_0/checkpoint-5800",  run_name="Optuna")
# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_val_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x0000027DA2B772B0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 27e37a07850, raw_cell="# Create the Trainer
#trainer = create_trainer(mod.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/OEM/Notebooks/COMPSCI764/Project/4.%20ReClor%20Baseline%20Training%20and%20Evaluation.ipynb#X43sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

  0%|          | 0/125 [00:00<?, ?it/s]

KeyboardInterrupt: 

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x0000027DA2B772B0>> (for post_run_cell), with arguments args (<ExecutionResult object at 27e37a07610, execution_count=56 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 27e37a07850, raw_cell="# Create the Trainer
#trainer = create_trainer(mod.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/OEM/Notebooks/COMPSCI764/Project/4.%20ReClor%20Baseline%20Training%20and%20Evaluation.ipynb#X43sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [19]:
# Create a study object and optimize the objective
study = optuna.create_study(direction='maximize')
#study.enqueue_trial({'model_name': 'microsoft/deberta-v3-base', 'learning_rate': 1.5807103066634623e-05, 'batch_size': 4, 'warmup_ratio': 0.5994150649377659, 'weight_decay': 0.12506835879573128, 'adam_beta1': 0.8136227307274486, 'adam_beta2': 0.9924116710027883, 'adam_epsilon': 1.9858068243318367e-07, 'lr_scheduler_type': 'cosine_with_restarts'})
study.enqueue_trial({'model_name': './MCQA-Combined/Optuna/trial_5/checkpoint-22865', 'learning_rate': 1.5807103066634623e-05, 'batch_size': 4, 'warmup_ratio': 0.5994150649377659, 'weight_decay': 0.12506835879573128, 'adam_beta1': 0.8136227307274486, 'adam_beta2': 0.9924116710027883, 'adam_epsilon': 1.9858068243318367e-07, 'lr_scheduler_type': 'cosine_with_restarts'})
study.optimize(objective, n_trials=20)


[I 2024-10-18 15:11:55,784] A new study created in memory with name: no-name-ea4bfe48-1cd2-49e9-baaa-1e5c2e900ae2


Current Trial 0 parameters: {'learning_rate': 1.5807103066634623e-05, 'batch_size': 4, 'warmup_ratio': 0.5994150649377659, 'weight_decay': 0.12506835879573128, 'adam_beta1': 0.8136227307274486, 'adam_beta2': 0.9924116710027883, 'adam_epsilon': 1.9858068243318367e-07, 'lr_scheduler_type': 'cosine_with_restarts'}


c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/34800 [00:00<?, ?it/s]

{'loss': 0.2007, 'grad_norm': 3.974223363911733e-05, 'learning_rate': 3.7509664515743714e-07, 'epoch': 0.43}
{'loss': 0.1361, 'grad_norm': 143.74293518066406, 'learning_rate': 7.539821453164645e-07, 'epoch': 0.86}


...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_accuracy': 0.935871743486974, 'eval_f1': 0.9359157700799514, 'eval_loss': 0.557083785533905, 'eval_runtime': 20.4528, 'eval_samples_per_second': 24.398, 'eval_steps_per_second': 6.112, 'epoch': 2.0}
Stopping training: No improvement in eval_accuracy for 1 epochs
{'train_runtime': 2016.148, 'train_samples_per_second': 69.013, 'train_steps_per_second': 17.261, 'train_loss': 0.16425138341969459, 'epoch': 2.0}


  0%|          | 0/125 [00:00<?, ?it/s]

Stopping training: No improvement in eval_accuracy for 1 epochs


[I 2024-10-19 08:55:00,648] Trial 19 finished with value: 0.935871743486974 and parameters: {'learning_rate': 6.169994646982436e-07, 'batch_size': 4, 'warmup_ratio': 0.9886008700272123, 'weight_decay': 0.045635541062629625, 'adam_beta1': 0.8867276898827927, 'adam_beta2': 0.9950703113403668, 'adam_epsilon': 7.987487123856168e-07, 'lr_scheduler_type': 'cosine'}. Best is trial 9 with value: 0.9418837675350702.


## Second Tuning

In [19]:
# Create the Trainer
check_point_path="./MCQA-Combined/Optuna2/trial_0/checkpoint-75540" # 58.6% ReClore Leaderboard
trainer = create_trainer(model_name=check_point_path,  run_name="Optuna3", num_train_epochs=10)

# Train the model
trainer.train()

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/11600 [00:00<?, ?it/s]

KeyboardInterrupt: 

## LeaderBoard File generation

In [23]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForMultipleChoice
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Load the tokenizer and model from the same checkpoint used during training
#check_point_path="./ReClor/Optuna3/trial_17/checkpoint-3480" # 59.5% ReClore Leaderboard
#check_point_path="./MCQA-Combined/Optuna2/trial_0/checkpoint-62950" # 57.10% ReClore Leaderboard
#check_point_path="./MCQA-Combined/Optuna2/trial_0/checkpoint-75540" # 58.6% ReClore Leaderboard
#check_point_path="./MCQA-Combined/Optuna2/trial_0/checkpoint-125900" # 55.0% ReClore Leaderboard

#check_point_path="./MCQA-Combined-9/Optuna-1/trial_0/checkpoint-36630" # 71.04% Validation - 58% ReClore Leaderboard
#check_point_path="./MCQA-Combined-9/Optuna-1/trial_0/checkpoint-48840" # 72.60% Validation - 57.40% ReClore Leaderboard
#check_point_path="./MCQA-Combined-9/Optuna-1/trial_0/checkpoint-61050" # % Validation - 57.90% ReClore Leaderboard
#check_point_path="./MCQA-Combined-9/Optuna-1/trial_0/checkpoint-97680" # 75.20% Validation - 59.6% ReClore Leaderboard
#check_point_path="./MCQA-Combined-9/Optuna-1/trial_4/checkpoint-12210" # 60.78% Validation - 54.2% ReClore Leaderboard
check_point_path="./MCQA-Combined-9/Optuna-1/trial_4/checkpoint-24420" # 63.46% Validation - 54.9% ReClore Leaderboard


tokenizer = AutoTokenizer.from_pretrained(check_point_path)
model = AutoModelForMultipleChoice.from_pretrained(check_point_path)

# Load test data
with open('./ReClor/reclor_data/test.json', 'r') as file:
    test_data = json.load(file)

class MultipleChoiceDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512, max_num_choices=5):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.max_num_choices = max_num_choices

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        context = item['context']
        question = item['question']
        options = item['answers']
        num_choices = len(options)
        # Pad options to have max_num_choices
        if num_choices < self.max_num_choices:
            options += [''] * (self.max_num_choices - num_choices)
        num_choices = self.max_num_choices

        # Combine question with each option
        second_sentences = [f"{question} {option}" for option in options]

        encoded = self.tokenizer(
            [context] * num_choices,  # Context repeated for each option
            second_sentences,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded['input_ids'],  # Do not squeeze
            'attention_mask': encoded['attention_mask'],  # Do not squeeze
            'labels': torch.tensor(0)  # Dummy label
        }

# Create dataset and DataLoader
test_dataset = MultipleChoiceDataset(test_data, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Prediction loop with tqdm for progress bar
predictions = []
model.eval()
for batch in tqdm(test_loader, desc="Predicting", unit="batch"):
    with torch.no_grad():
        outputs = model(
            input_ids=batch['input_ids'],  # Do not squeeze
            attention_mask=batch['attention_mask']  # Do not squeeze
        )
    pred = torch.argmax(outputs.logits, dim=1)
    predictions.extend(pred.cpu().numpy())

# Create ID string mapping to predictions
predicted_labels = {f"test_{i}": int(pred) for i, pred in enumerate(predictions)}

# Save predictions to .npy file
np.save('test_predictions.npy', np.array(predictions))

print("Predictions saved successfully.")


Predicting: 100%|██████████| 1000/1000 [17:47<00:00,  1.07s/batch]

Predictions saved successfully.


In [20]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForMultipleChoice
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Load the tokenizer and model from the same checkpoint used during training
tokenizer = AutoTokenizer.from_pretrained("./ReClor/Optuna/trial_0/checkpoint-5800")
model = AutoModelForMultipleChoice.from_pretrained("./ReClor/Optuna/trial_0/checkpoint-5800")

# Load test data
with open('./ReClor/reclor_data/test.json', 'r') as file:
    test_data = json.load(file)

class MultipleChoiceDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512, max_num_choices=5):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.max_num_choices = max_num_choices

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        context = item['context']
        question = item['question']
        options = item['answers']
        num_choices = len(options)
        # Pad options to have max_num_choices
        if num_choices < self.max_num_choices:
            options += [''] * (self.max_num_choices - num_choices)
        num_choices = self.max_num_choices

        # Combine question with each option
        second_sentences = [f"{question} {option}" for option in options]

        encoded = self.tokenizer(
            [context] * num_choices,  # Context repeated for each option
            second_sentences,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded['input_ids'],  # Do not squeeze
            'attention_mask': encoded['attention_mask'],  # Do not squeeze
            'labels': torch.tensor(0)  # Dummy label
        }

# Create dataset and DataLoader
test_dataset = MultipleChoiceDataset(test_data, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Prediction loop with tqdm for progress bar
predictions = []
model.eval()
for batch in tqdm(test_loader, desc="Predicting", unit="batch"):
    with torch.no_grad():
        outputs = model(
            input_ids=batch['input_ids'],  # Do not squeeze
            attention_mask=batch['attention_mask']  # Do not squeeze
        )
    pred = torch.argmax(outputs.logits, dim=1)
    predictions.extend(pred.cpu().numpy())

# Create ID string mapping to predictions
predicted_labels = {f"test_{i}": int(pred) for i, pred in enumerate(predictions)}

# Save predictions to .npy file
np.save('test_predictions.npy', np.array(predictions))

print("Predictions saved successfully.")


Predicting: 100%|██████████| 1000/1000 [16:47<00:00,  1.01s/batch]

Predictions saved successfully.


In [15]:
print(len(test_data))  # This will print the number of entries in the test.json
print(len(test_dataset))  # This will also show the number of entries loaded into your dataset class


1000
1000


In [15]:
import json
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForMultipleChoice
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch.nn as nn


# Load the base DeBERTa model
model = AutoModel.from_pretrained("microsoft/deberta-base-mnli")
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base-mnli")



class DeBERTaForMultipleChoice(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.deberta = base_model
        self.classifier = nn.Linear(self.deberta.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        batch_size = input_ids.size(0)
        num_choices = input_ids.size(1)
        seq_length = input_ids.size(2)

        # Flatten the batch and choice dimensions
        flat_input_ids = input_ids.view(batch_size * num_choices, seq_length)
        flat_attention_mask = attention_mask.view(batch_size * num_choices, seq_length)

        outputs = self.deberta(input_ids=flat_input_ids, attention_mask=flat_attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [batch_size*num_choices, hidden_size]

        logits = self.classifier(pooled_output)  # [batch_size*num_choices, 1]

        # Reshape logits back to [batch_size, num_choices]
        logits = logits.view(batch_size, num_choices)
        return logits


# Initialize the custom multiple choice model
mc_model = DeBERTaForMultipleChoice(model)

# Load test data
with open('./ReClor/reclor_data/test.json', 'r') as file:
    test_data = json.load(file)

class MultipleChoiceDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        context = item['context']
        question = item['question']
        options = item['answers']
        encoded = self.tokenizer(
            [context] * len(options),  # Context repeated for each option
            options,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded['input_ids'],  # Keep the dimensions as is
            'attention_mask': encoded['attention_mask'],
            'labels': torch.tensor(item.get('question_type', 0))  # Dummy labels
        }


# Create dataset and DataLoader
test_dataset = MultipleChoiceDataset(test_data, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Prediction loop with tqdm for progress bar
mc_model.eval()
predictions = []
for batch in tqdm(test_loader, desc="Predicting", unit="batch"):
    with torch.no_grad():
        logits = mc_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
    pred = torch.argmax(logits, dim=1)
    predictions.extend(pred.numpy())


# Create ID string mapping to predictions
predicted_labels = {f"test_{i}": pred for i, pred in enumerate(predictions)}

# Save predictions to .npy file
np.save('test_predictions.npy', np.array(predictions))

print("Predictions saved successfully.")


Predicting:  66%|██████▌   | 662/1000 [09:53<05:02,  1.12batch/s]


KeyboardInterrupt: 